# Traits

**Advantages** of traits

* Can be defined after the type is declared (unlike a supertype).
* Don't have to be created up-front, so types can be added later (unlike a Union).
* Otherwise-unrelated types (unlike a supertype) can be used.

Traits have a few **key parts**:

* Trait types: the different traits a type can have.
* Trait function: what traits a type has.
* Trait dispatch: using the traits.

To understand how traits work, it is important to understand the type of types in Julia. Types are values, so they have a type themselves: `DataType`. However, they also have the special pseudo-supertype `Type`, so a type `T` acts like `T<:Type{T}`.

In [ ]:
typeof(String) == DataType

In [ ]:
String isa Type{String}

In [ ]:
String isa Type{<:AbstractString}

### Trait type

This is the type that is used to attribute a particular trait.

In [ ]:
abstract type StatQualia end

struct Continuous <: StatQualia end
struct Categorical <: StatQualia end

### Trait function

The trait function takes a type as input, and returns an instance of the trait type. We use the trait function to declare what traits a particular type has. For example, we can say things like floats are continuous, booleans are categorical, etc.

In [ ]:
statqualia(::Type{<:AbstractFloat}) = Continuous()
statqualia(::Type{<:Bool}) = Categorical()
statqualia(::Type{<:AbstractString}) = Categorical()

### Using Traits

To use a trait we need to re-dispatch upon it. This is where we take the type of an input, and invoke the trait function on it to get objects of the trait type, then dispatch on those.

In [ ]:
using LinearAlgebra

# This is the trait re-dispatch
bounds(xs::AbstractVector{T}) where T = bounds(statqualia(T), xs)

# Dispatch on the trait
bounds(::Continuous, xs) = extrema(xs)
bounds(::Categorical, xs) = unique(xs)

In [ ]:
bounds([false, true, false, false, true])

In [ ]:
bounds([1.2, 4.8, 3.1, 11.9])

In [ ]:
@code_warntype debuginfo=:none bounds([1.2, 4.8, 3.1, 11.9])

In [ ]:
@code_native debuginfo=:none bounds([1.2, 4.8, 3.1, 11.9])

In [ ]:
@code_native debuginfo=:none extrema([1.2, 4.8, 3.1, 11.9])

## Extending and using Traits

### Indicate trait for new custom type

In [ ]:
abstract type Color end
struct Red <: Color end
struct Blue <: Color end
struct Green <: Color end

statqualia(::Type{Color}) = Categorical()

In [ ]:
bounds([Red(), Blue(), Blue(), Green()])

In [ ]:
struct Measurement
    x::Float64
end

statqualia(::Type{Measurement}) = Continuous()

# make extrema work for our type
Base.isless(a::Measurement, b::Measurement) = a.x < b.x

In [ ]:
ms = Measurement.(rand(3))

In [ ]:
bounds(ms)

### Extend current set of trait options

In [ ]:
# define new trait options
struct Ordinal <: StatQualia end

# define trait functions
statqualia(::Type{<:Integer}) = Ordinal()

# use new traits
bounds(::Ordinal, xs) = extrema(xs)

In [ ]:
bounds([1,2,3,4,5])

Resources:
* "Invented" by Tim Holy in this [github issue](https://github.com/JuliaLang/julia/issues/2345#issuecomment-54537633).
* See https://github.com/mauro3/Traits.jl#dispatch-on-traits for a detailed explanation and [SimpleTraits.jl](https://github.com/mauro3/SimpleTraits.jl) for a convenience implementation.
* The above is based on this blog post https://invenia.github.io/blog/2019/11/06/julialang-features-part-2/